<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<b>Little Demo: Point source CMD (Color-Magnitude Diagram)</b> <br>
Contact author(s): Jeff Carlin and Yumi Choi <br>
Last verified to run: 2024-01-25 <br>
LSST Science Pipelines version: Weekly 2023_47 <br>
Container Size: medium

Create a stellar color-magnitude diagram (CMD) and color-color diagram(s).

**Option** to create the same figures in the Portal in Section 5. <br>

## 1. Set up

Import packages.

In [ ]:
import matplotlib.pyplot as plt
from lsst.rsp import get_tap_service, retrieve_query

Instantiate the TAP service.

In [ ]:
service = get_tap_service("tap")

Explore the DP0.2 schema.

In [ ]:
dp02_schema_name = 'dp02_dc2_catalogs'

In [ ]:
query = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = '" \
        + dp02_schema_name + "' order by table_index ASC"

results = service.search(query).to_table()
results

Get the columns available for the dp02_dc2_catalogs.Object table.

In [ ]:
results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp02_dc2_catalogs.Object'")
results.to_table().to_pandas()

Clean up

In [ ]:
del results

## 2. TAP query

Define the TAP query to retrieve point-like Objects in a 2-degree radius region. The Object catalog contains objects detected in deep coadds.

In [ ]:
query = "SELECT scisql_nanojanskyToAbMag(g_psfFlux) as gmag, " + \
        "scisql_nanojanskyToAbMag(r_psfFlux) as rmag, " + \
        "scisql_nanojanskyToAbMag(i_psfFlux) as imag, " + \
        "scisql_nanojanskyToAbMag(z_psfFlux) as zmag, " + \
        "scisql_nanojanskyToAbMag(y_psfFlux) as ymag " + \
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', 55.75, -32.27, 2.0)) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND i_extendedness = 0 AND z_extendedness = 0 " + \
        "AND i_psfFlux/i_psfFluxErr > 5 " + \
        "AND z_psfFlux/z_psfFluxErr > 5 " + \
        "AND y_psfFlux/y_psfFluxErr > 5 "

Execute the query, and store the results in an `astropy` table.

In [ ]:
results = service.search(query).to_table()

Uncomment the following cell to view the table.

In [ ]:
# results

## 3. Create CMDs and color-color diagrams

Plot i-band magnitude vs. g-r color, then i-z color, as scatter plots.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))

ax[0].plot(results['gmag'] - results['rmag'], results['imag'], 
           'o', ms=2, color='gray', alpha=0.2, mew=0)
ax[0].set_xlabel('g-r color', fontsize=16)
ax[0].set_ylabel('i magnitude', fontsize=16)
ax[0].set_xlim(-0.7, 2)
ax[0].set_ylim(26, 16)

ax[1].plot(results['imag'] - results['zmag'], results['imag'],
           'o', ms=2, color='grey', alpha=0.2, mew=0)
ax[1].set_xlabel('i-z color', fontsize=16)
ax[1].set_xlim(-0.7, 2)
ax[1].set_ylim(26, 16)

plt.tight_layout()
plt.show()

Plot color-color diagrams for (r-i) vs. (g-r), (r-z) vs. (g-r), and (z-y) vs. (i-z).

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(11, 4))

ax[0].plot(results['gmag'] - results['rmag'], results['rmag'] - results['imag'],
           'o', ms=2, color='red', alpha=0.1, mew=0)
ax[0].set_xlabel('g-r color', fontsize=16)
ax[0].set_ylabel('r-i color', fontsize=16)
ax[0].set_ylim(-0.7, 3.5)
ax[0].set_xlim(-0.7, 2.5)

ax[1].plot(results['gmag'] - results['rmag'], results['rmag'] - results['zmag'],
           'o', ms=2, color='red', alpha=0.1, mew=0)
ax[1].set_xlabel('g-r color', fontsize=16)
ax[1].set_ylabel('r-z color', fontsize=16)
ax[1].set_ylim(-0.7, 3.5)
ax[1].set_xlim(-0.7, 2.5)

ax[2].plot(results['imag'] - results['zmag'], results['zmag'] - results['ymag'],
         'o', ms=2, color='red', alpha=0.1, mew=0)
ax[2].set_xlabel('i-z color', fontsize=16)
ax[2].set_ylabel('z-y color', fontsize=16)
ax[2].set_ylim(-0.7, 2.0)
ax[2].set_xlim(-0.7, 2.5)

plt.tight_layout()
plt.show()

## 4. Examine red point sources (likely cool dwarfs)

### Select candidate MLT dwarfs

Try a color-color selection to isolate the coolest (late-M-, L-, and T-type?) dwarfs.

In [ ]:
cooldwarfs = (results['imag']-results['zmag'] > 1.0) & (results['zmag']-results['ymag'] > 0.6)

Highlight the selected objects in the CMDs.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))

ax[0].plot(results['zmag'] - results['ymag'], results['imag'],
           'o', ms=2, color='grey', alpha=0.2, mew=0)
ax[0].plot(results[cooldwarfs]['zmag'] - results[cooldwarfs]['ymag'], 
           results[cooldwarfs]['imag'],
           'o', ms=2, color='red', alpha=0.5, mew=0)
ax[0].set_xlabel('z-y color', fontsize=16)
ax[0].set_ylabel('i magnitude', fontsize=16)
ax[0].set_xlim(-0.7, 2)
ax[0].set_ylim(26, 16)

ax[1].plot(results['imag'] - results['zmag'], results['imag'],
         'o', ms=2, color='grey', alpha=0.1, mew=0)
ax[1].plot(results[cooldwarfs]['imag'] - results[cooldwarfs]['zmag'],
           results[cooldwarfs]['imag'],
           'o', ms=2, color='red', alpha=0.5, mew=0)
ax[1].set_xlabel('i-z color', fontsize=16)
ax[1].set_ylabel('i magnitude', fontsize=16)
ax[1].set_xlim(-0.7, 2)
ax[1].set_ylim(26, 16)

plt.show()

## 5. Repeat in the Portal

**Option** to execute the same query in the Portal.

5.1. Go to <a href="https://data.lsst.cloud/">data.lsst.cloud</a> in a new browser tab, and enter the Portal aspect.

5.2. In the Portal, next to “View” at the upper right, switch from the default “UI assisted” to “Edit ADQL”.

5.3. Copy and paste the following into the "ADQL Query" box.
```
SELECT scisql_nanojanskyToAbMag(g_psfFlux) as gmag,
scisql_nanojanskyToAbMag(r_psfFlux) as rmag,
scisql_nanojanskyToAbMag(i_psfFlux) as imag,
scisql_nanojanskyToAbMag(z_psfFlux) as zmag,
scisql_nanojanskyToAbMag(y_psfFlux) as ymag
FROM dp02_dc2_catalogs.Object
WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),
               CIRCLE('ICRS', 55.75, -32.27, 2.0)) = 1
AND detect_isPrimary = 1
AND i_extendedness = 0 AND z_extendedness = 0
AND i_psfFlux/i_psfFluxErr > 5
AND z_psfFlux/z_psfFluxErr > 5
AND y_psfFlux/y_psfFluxErr > 5
```

5.4. At lower left, change the Row Limit to 75000 from the default 50000 and then click "Search".
The Portal will transfer to the results view, which has two components (xy plot and table).

5.5. View the default xy plot, a heatmap (2-d histogram) of g-band magnitude vs. r-band magnitude (i.e., the first two columns of data returned).

5.6. Alter the default xy plot to be a CMD. Click on the double-gears icon at upper right to open the "Plot Parameters" pop-up window.

5.7. Change "X:" to "imag-zmag" and "Y:" to "imag", click "Apply", then click "Close". The Portal does not currently offer the capability to log scale the heatmap bin values.


<img src="figures/ucd-cmd-portal-screenshot.png" alt="Portal screenshot with final CMD." class="bg-primary" width="500px">